In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn


In [26]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
# Convert y to int (0/1) instead of bool to avoid ONNX conversion issues
y = data['checked'].astype(int)
X = data.drop(['Ja', 'Nee', 'checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [27]:
# Define sensitive features to check for bias
SENSITIVE_FEATURES = ['persoon_geslacht_vrouw', 'pla_hist_pla_categorie_doelstelling_16']
BIAS_THRESHOLD = 0.03  # Consider bias significant if accuracy difference > 3%


In [28]:
# ============================================================================
# BAD MODEL: RandomForest without any preprocessing
# ============================================================================
print("=" * 70)
print("Training BAD MODEL (RandomForest - No Preprocessing)")
print("=" * 70)

bad_classifier = RandomForestClassifier(random_state=0)
bad_pipeline = Pipeline(steps=[('classification', bad_classifier)])

# Train the bad model
bad_pipeline.fit(X_train, y_train)

# Evaluate the bad model
y_pred_bad = bad_pipeline.predict(X_test)
bad_accuracy = accuracy_score(y_test, y_pred_bad)
print(f'\nOverall Accuracy of BAD model: {bad_accuracy:.4f}')

# Check bias on sensitive features for bad model
print("\nBias Analysis for BAD Model:")
for feature in SENSITIVE_FEATURES:
    if feature in X_test.columns:
        accuracies = {}
        for value in [0.0, 1.0]:
            mask = X_test[feature] == value
            if mask.sum() > 0:
                group_pred = y_pred_bad[mask]
                group_true = y_test[mask]
                group_accuracy = accuracy_score(group_true, group_pred)
                accuracies[value] = group_accuracy
                print(f"  {feature} = {value}: Accuracy = {group_accuracy:.4f} (n={mask.sum()})")
        
        # Calculate and display delta
        if len(accuracies) == 2:
            delta = accuracies[1.0] - accuracies[0.0]
            print(f"  → Delta (1.0 - 0.0): {delta:+.4f}")


Training BAD MODEL (RandomForest - No Preprocessing)

Overall Accuracy of BAD model: 0.8766

Bias Analysis for BAD Model:
  persoon_geslacht_vrouw = 0.0: Accuracy = 0.8776 (n=16673)
  persoon_geslacht_vrouw = 1.0: Accuracy = 0.8755 (n=15827)
  → Delta (1.0 - 0.0): -0.0021
  pla_hist_pla_categorie_doelstelling_16 = 0.0: Accuracy = 0.8154 (n=14603)
  pla_hist_pla_categorie_doelstelling_16 = 1.0: Accuracy = 0.9227 (n=16102)
  → Delta (1.0 - 0.0): +0.1073


In [29]:
# ============================================================================
# GOOD MODEL: AdaBoost with DecisionTree base estimator
# ============================================================================
print("=" * 70)
print("Training GOOD MODEL (AdaBoost with DecisionTree base estimator)")
print("=" * 70)

# Create AdaBoostClassifier with DecisionTreeClassifier(max_depth=2) as base estimator
good_classifier = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=2),
    n_estimators=100,
    learning_rate=1.0,
    random_state=0
)
good_pipeline = Pipeline(steps=[('classification', good_classifier)])

# Train the good model
good_pipeline.fit(X_train, y_train)

# Evaluate the good model
y_pred_good = good_pipeline.predict(X_test)
good_accuracy = accuracy_score(y_test, y_pred_good)
print(f'\nOverall Accuracy of GOOD model: {good_accuracy:.4f}')

# Check bias on sensitive features for good model
print("\nBias Analysis for GOOD Model:")
for feature in SENSITIVE_FEATURES:
    if feature in X_test.columns:
        accuracies = {}
        for value in [0.0, 1.0]:
            mask = X_test[feature] == value
            if mask.sum() > 0:
                group_pred = y_pred_good[mask]
                group_true = y_test[mask]
                group_accuracy = accuracy_score(group_true, group_pred)
                accuracies[value] = group_accuracy
                print(f"  {feature} = {value}: Accuracy = {group_accuracy:.4f} (n={mask.sum()})")
        
        # Calculate and display delta
        if len(accuracies) == 2:
            delta = accuracies[1.0] - accuracies[0.0]
            print(f"  → Delta (1.0 - 0.0): {delta:+.4f}")

Training GOOD MODEL (AdaBoost with DecisionTree base estimator)

Overall Accuracy of GOOD model: 0.9460

Bias Analysis for GOOD Model:
  persoon_geslacht_vrouw = 0.0: Accuracy = 0.9455 (n=16673)
  persoon_geslacht_vrouw = 1.0: Accuracy = 0.9465 (n=15827)
  → Delta (1.0 - 0.0): +0.0010
  pla_hist_pla_categorie_doelstelling_16 = 0.0: Accuracy = 0.9231 (n=14603)
  pla_hist_pla_categorie_doelstelling_16 = 1.0: Accuracy = 0.9630 (n=16102)
  → Delta (1.0 - 0.0): +0.0400


In [30]:
# ============================================================================
# Convert BAD model to ONNX
# ============================================================================
print("\n" + "=" * 70)
print("Converting BAD model to ONNX")
print("=" * 70)

bad_onnx_model = convert_sklearn(
    bad_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Check accuracy of converted bad model
bad_sess = rt.InferenceSession(bad_onnx_model.SerializeToString())
y_pred_bad_onnx = bad_sess.run(None, {'X': X_test.values.astype(np.float32)})
bad_accuracy_onnx = accuracy_score(y_test, y_pred_bad_onnx[0])
print(f'Accuracy of BAD ONNX model: {bad_accuracy_onnx:.4f}')

# Save bad model
onnx.save(bad_onnx_model, "model/model_2.onnx")
print("Saved BAD model to: model/model_2.onnx")



Converting BAD model to ONNX
Accuracy of BAD ONNX model: 0.8766
Saved BAD model to: model/model_2.onnx


In [ ]:
# ============================================================================
# Convert GOOD model to ONNX
# ============================================================================
print("\n" + "=" * 70)
print("Converting GOOD model to ONNX")
print("=" * 70)

# Use the correct number of features (after removing sensitive features if bias was detected)
good_onnx_model = convert_sklearn(
    good_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# Check accuracy of converted good model
good_sess = rt.InferenceSession(good_onnx_model.SerializeToString())
y_pred_good_onnx = good_sess.run(None, {'X': X_test.values.astype(np.float32)})
good_accuracy_onnx = accuracy_score(y_test, y_pred_good_onnx[0])
print(f'Accuracy of GOOD ONNX model: {good_accuracy_onnx:.4f}')

# Save good model
onnx.save(good_onnx_model, "model/model_1.onnx")
print("Saved GOOD model to: model/model_1.onnx")



Converting GOOD model to ONNX


ValueError: Found input variables with inconsistent numbers of samples: [32500, 130000]

In [33]:
# ============================================================================
# Summary Comparison
# ============================================================================
print("\n" + "=" * 70)
print("MODEL COMPARISON SUMMARY")
print("=" * 70)
print(f"\nBAD Model (RandomForest, no preprocessing):")
print(f"  Overall Accuracy: {bad_accuracy:.4f}")

print(f"\nGOOD Model (AdaBoost, post-processing with threshold per group):")
print(f"  Overall Accuracy: {good_accuracy:.4f}")

print(f"\nDifference: {good_accuracy - bad_accuracy:+.4f}")

# Verify saved models can be loaded
print("\n" + "=" * 70)
print("Verifying saved models...")
print("=" * 70)

bad_loaded = rt.InferenceSession("model/model_2.onnx")
good_loaded = rt.InferenceSession("model/model_1.onnx")

y_pred_bad_loaded = bad_loaded.run(None, {'X': X_test.values.astype(np.float32)})
# Good model expects features without sensitive ones (X_test_fair)
y_pred_good_loaded = good_loaded.run(None, {'X': X_test_fair.values.astype(np.float32)})

bad_loaded_acc = accuracy_score(y_test, y_pred_bad_loaded[0])
good_loaded_acc = accuracy_score(y_test, y_pred_good_loaded[0])

print(f"BAD model loaded accuracy: {bad_loaded_acc:.4f}")
print(f"GOOD model loaded accuracy: {good_loaded_acc:.4f}")
print("\n✓ Both models saved and verified successfully!")



MODEL COMPARISON SUMMARY

BAD Model (RandomForest, no preprocessing):
  Overall Accuracy: 0.9058

GOOD Model (AdaBoost, post-processing with threshold per group):
  Overall Accuracy: 0.9526

Difference: +0.0468

Verifying saved models...
BAD model loaded accuracy: 0.9058
GOOD model loaded accuracy: 0.9526

✓ Both models saved and verified successfully!
